In [20]:
# getting quotes with apimoex library
# apimoex should be installed — https://pypi.org/project/apimoex/
# installation through terminal — pip install apimoex

import requests
import apimoex
import pandas as pd
from datetime import date, timedelta

class MoexQuoteProvider():
    def __init__(
        self,
    ): pass
    
    def getQuotes(
        self,
        ticker:str,
        startDate:date = date(date.today().year-1,date.today().month,date.today().day),
        endDate:date = date.today(),
        returnClose=False
    ):
        with requests.Session() as session:
            data = apimoex.get_board_history(session, ticker)
            moexDf = pd.DataFrame(data)
            moexDf.set_index('TRADEDATE', inplace=True)
            moexDf.index=pd.to_datetime(moexDf.index)
            moexDf.insert(0,'SECID',ticker)
            moexDf.drop(['VALUE','VOLUME'], axis=1, inplace=True)
        if returnClose==False:
            return moexDf.loc[startDate:endDate]
        else:
            return moexDf['CLOSE'].iloc[-1]
    
    
sampleQuoteObj=MoexQuoteProvider()
print(sampleQuoteObj.getQuotes('SBER',date(2020,10,9),date(2022,10,9),returnClose=True))
sampleMoexDf = sampleQuoteObj.getQuotes('SBER')
print(sampleMoexDf)


101.5
           SECID BOARDID   CLOSE
TRADEDATE                       
2021-10-11  SBER    TQBR  387.60
2021-10-12  SBER    TQBR  377.79
2021-10-13  SBER    TQBR  378.80
2021-10-14  SBER    TQBR  376.09
2021-10-15  SBER    TQBR  371.82
...          ...     ...     ...
2022-10-03  SBER    TQBR  114.35
2022-10-04  SBER    TQBR  110.55
2022-10-05  SBER    TQBR  108.51
2022-10-06  SBER    TQBR  108.24
2022-10-07  SBER    TQBR  101.50

[251 rows x 3 columns]


In [9]:
# getting quotes with finec.moex library
# finec.moex should be also installed — https://github.com/epogrebnyak/finec
# installation through terminal — pip install git+https://github.com/epogrebnyak/finec.git --user

from finec.moex import Stock

class FinecQuoteProvider():
    def __init__(
        self,
    ): pass
    
    def getQuotes(
        self,
        ticker:str,
        startDate:date = date(date.today().year-1,date.today().month,date.today().day),
        endDate:date = date.today(),
        returnClose=False
    ):
        finecDf = Stock(ticker).get_history()
        finecDf.index=pd.to_datetime(finecDf.index)
        finecDf.drop(['WAPRICE','NUMTRADES','VALUE','VOLUME'], axis=1, inplace=True)
        if returnClose==False:
            return finecDf.loc[startDate:endDate]
        else:
            return finecDf['CLOSE'].iloc[-1]
    
    
sampleQuoteObj=FinecQuoteProvider()
print(sampleQuoteObj.getQuotes('SBER',date(2020,10,9),date(2022,10,9),returnClose=True))
sampleFinecDf = sampleQuoteObj.getQuotes('SBER')
print(sampleFinecDf)

101.5
           SECID BOARDID   CLOSE
TRADEDATE                       
2021-10-11  SBER    TQBR  387.60
2021-10-12  SBER    TQBR  377.79
2021-10-13  SBER    TQBR  378.80
2021-10-14  SBER    TQBR  376.09
2021-10-15  SBER    TQBR  371.82
...          ...     ...     ...
2022-10-03  SBER    TQBR  114.35
2022-10-04  SBER    TQBR  110.55
2022-10-05  SBER    TQBR  108.51
2022-10-06  SBER    TQBR  108.24
2022-10-07  SBER    TQBR  101.50

[251 rows x 3 columns]


In [21]:
# cheching whether two methods work identically
sampleFinecDf.equals(sampleMoexDf)

True

In [ ]:
# getting quotes from finam by extracting data from downloaded csv file

In [24]:
from datetime import datetime
from urllib.parse import urlencode
from urllib.request import urlopen
import io
import requests
import pandas as pd

class FinamQuoteProvider():
    def __init__(
        self,
    ): pass

    def getQuotes(
            self,
            ticker:str,
            startDate:date = date(date.today().year-1,date.today().month,date.today().day),
            endDate:date = date.today(),
            returnClose=False
        ):
            start = startDate.strftime("%d.%m.%Y")
            end = endDate.strftime("%d.%m.%Y")          
            periods={'tick': 1, 'min': 2, '5min': 3, '10min': 4, '15min': 5, '30min': 6, 'hour': 7, 'daily': 8, 'week': 9, 'month': 10}
            period = 8 #daily qoutes
            
            # every instrument in finam has its own numeric id, but the list may be outdated
            tickers={'OZON':2179435,'ABRD':82460,'AESL':181867,'AFKS':19715,'AFLT':29,'AGRO':399716,'AKRN':17564,'ALBK':82616,'ALNU':81882,'ALRS':81820,'AMEZ':20702,'APTK':13855,'AQUA':35238,'ARMD':19676,'ARSA':19915,'ASSB':16452,'AVAN':82843,'AVAZ':39,'AVAZP':40,'BANE':81757,'BANEP':81758,'BGDE':175840,'BISV':35242,'BISVP':35243,'BLNG':21078,'BRZL':81901,'BSPB':20066,'CBOM':420694,'CHEP':20999,'CHGZ':81933,'CHKZ':21000,'CHMF':16136,'CHMK':21001,'CHZN':19960,'CLSB':16712,'CLSBP':16713,'CNTL':21002,'CNTLP':81575,'DASB':16825,'DGBZ':17919,'DIOD':35363,'DIXY':18564,'DVEC':19724,'DZRD':74744,'DZRDP':74745,'ELTZ':81934,'ENRU':16440,'EPLN':451471,'ERCO':81935,'FEES':20509,'FESH':20708,'FORTP':82164,'GAZA':81997,'GAZAP':81998,'GAZC':81398,'GAZP':16842,'GAZS':81399,'GAZT':82115,'GCHE':20125,'GMKN':795,'GRAZ':16610,'GRNT':449114,'GTLC':152876,'GTPR':175842,'GTSS':436120,'HALS':17698,'HIMC':81939,'HIMCP':81940,'HYDR':20266,'IDJT':388276,'IDVP':409486,'IGST':81885,'IGST03':81886,'IGSTP':81887,'IRAO':20516,'IRGZ':9,'IRKT':15547,'ISKJ':17137,'JNOS':15722,'JNOSP':15723,'KAZT':81941,'KAZTP':81942,'KBSB':19916,'KBTK':35285,'KCHE':20030,'KCHEP':20498,'KGKC':83261,'KGKCP':152350,'KLSB':16329,'KMAZ':15544,'KMEZ':22525,'KMTZ':81903,'KOGK':20710,'KRKN':81891,'KRKNP':81892,'KRKO':81905,'KRKOP':81906,'KROT':510,'KROTP':511,'KRSB':20912,'KRSBP':20913,'KRSG':15518,'KSGR':75094,'KTSB':16284,'KTSBP':16285,'KUBE':522,'KUNF':81943,'KUZB':83165,'KZMS':17359,'KZOS':81856,'KZOSP':81857,'LIFE':74584,'LKOH':8,'LNTA':385792,'LNZL':21004,'LNZLP':22094,'LPSB':16276,'LSNG':31,'LSNGP':542,'LSRG':19736,'LVHK':152517,'MAGE':74562,'MAGEP':74563,'MAGN':16782,'MERF':20947,'MFGS':30,'MFGSP':51,'MFON':152516,'MGNT':17086,'MGNZ':20892,'MGTS':12984,'MGTSP':12983,'MGVM':81829,'MISB':16330,'MISBP':16331,'MNFD':80390,'MOBB':82890,'MOEX':152798,'MORI':81944,'MOTZ':21116,'MRKC':20235,'MRKK':20412,'MRKP':20107,'MRKS':20346,'MRKU':20402,'MRKV':20286,'MRKY':20681,'MRKZ':20309,'MRSB':16359,'MSNG':6,'MSRS':16917,'MSST':152676,'MSTT':74549,'MTLR':21018,'MTLRP':80745,'MTSS':15523,'MUGS':81945,'MUGSP':81946,'MVID':19737,'NAUK':81992,'NFAZ':81287,'NKHP':450432,'NKNC':20100,'NKNCP':20101,'NKSH':81947,'NLMK':17046,'NMTP':19629,'NNSB':16615,'NNSBP':16616,'NPOF':81858,'NSVZ':81929,'NVTK':17370,'ODVA':20737,'OFCB':80728,'OGKB':18684,'OMSH':22891,'OMZZP':15844,'OPIN':20711,'OSMP':21006,'OTCP':407627,'PAZA':81896,'PHOR':81114,'PHST':19717,'PIKK':18654,'PLSM':81241,'PLZL':17123,'PMSB':16908,'PMSBP':16909,'POLY':175924,'PRFN':83121,'PRIM':17850,'PRIN':22806,'PRMB':80818,'PRTK':35247,'PSBR':152320,'QIWI':181610,'RASP':17713,'RBCM':74779,'RDRB':181755,'RGSS':181934,'RKKE':20321,'RLMN':152677,'RLMNP':388313,'RNAV':66644,'RODNP':66693,'ROLO':181316,'ROSB':16866,'ROSN':17273,'ROST':20637,'RSTI':20971,'RSTIP':20972,'RTGZ':152397,'RTKM':7,'RTKMP':15,'RTSB':16783,'RTSBP':16784,'RUAL':414279,'RUALR':74718,'RUGR':66893,'RUSI':81786,'RUSP':20712,'RZSB':16455,'SAGO':445,'SAGOP':70,'SARE':11,'SAREP':24,'SBER':3,'SBERP':23,'SELG':81360,'SELGP':82610,'SELL':21166,'SIBG':436091,'SIBN':2,'SKYC':83122,'SNGS':4,'SNGSP':13,'STSB':20087,'STSBP':20088,'SVAV':16080,'SYNG':19651,'SZPR':22401,'TAER':80593,'TANL':81914,'TANLP':81915,'TASB':16265,'TASBP':16266,'TATN':825,'TATNP':826,'TGKA':18382,'TGKB':17597,'TGKBP':18189,'TGKD':18310,'TGKDP':18391,'TGKN':18176,'TGKO':81899,'TNSE':420644,'TORS':16797,'TORSP':16798,'TRCN':74561,'TRMK':18441,'TRNFP':1012,'TTLK':18371,'TUCH':74746,'TUZA':20716,'UCSS':175781,'UKUZ':20717,'UNAC':22843,'UNKL':82493,'UPRO':18584,'URFD':75124,'URKA':19623,'URKZ':82611,'USBN':81953,'UTAR':15522,'UTII':81040,'UTSY':419504,'UWGN':414560,'VDSB':16352,'VGSB':16456,'VGSBP':16457,'VJGZ':81954,'VJGZP':81955,'VLHZ':17257,'VRAO':20958,'VRAOP':20959,'VRSB':16546,'VRSBP':16547,'VSMO':15965,'VSYD':83251,'VSYDP':83252,'VTBR':19043,'VTGK':19632,'VTRS':82886,'VZRZ':17068,'VZRZP':17067,'WTCM':19095,'WTCMP':19096,'YAKG':81917,'YKEN':81766,'YKENP':81769,'YNDX':388383,'YRSB':16342,'YRSBP':16343,'ZHIV':181674,'ZILL':81918,'ZMZN':556,'ZMZNP':603,'ZVEZ':82001}
            finam_url = "http://export.finam.ru/"
            market = 0 # identifies the market (shares, bonds, currency etc.), for shares any number works
            start_date = datetime.strptime(start, "%d.%m.%Y").date()
            start_date_rev=datetime.strptime(start, '%d.%m.%Y').strftime('%Y%m%d')
            end_date = datetime.strptime(end, "%d.%m.%Y").date()
            end_date_rev=datetime.strptime(end, '%d.%m.%Y').strftime('%Y%m%d')
            params = urlencode([
                                ('market', market), # market
                                ('em', tickers[ticker]), #finam id for the instrument taken from the list "tickers"
                                ('code', ticker), 
                                ('apply',0), # unidentified parameter
                                ('df', start_date.day), # day number for the start date (1-31)
                                ('mf', start_date.month - 1), #month number for the start date (0-11)
                                ('yf', start_date.year),
                                ('from', start_date), 
                                ('dt', end_date.day), # day number for the start date (1-31)
                                ('mt', end_date.month - 1), #month number for the start date (0-11)
                                ('yt', end_date.year), 
                                ('to', end_date), 
                                ('p', period), 
                                ('f', ticker+"_" + start_date_rev + "_" + end_date_rev), # filename
                                ('e', ".csv"), # file extension
                                ('cn', ticker),
                                ('dtf', 1), # date format, for more info - https://www.finam.ru/profile/moex-akcii/sberbank/export/
                                ('tmf', 1), # time format
                                ('MSOR', 0), #candle (0 - open; 1 - close)
                                ('mstime', "on"), # Moscow time
                                ('mstimever', 1), # correction for timezone
                                ('sep', 1), # fields separator (1 - ",", 2 - ".", 3 - ";", 4 - "tab", 5 - "space")
                                ('sep2', 1), #digits separator
                                ('datf', 1), # data format (6 possible choices)
                                ('at', 1)]) # column headers 
            url = finam_url + ticker+"_" + start_date_rev + "_" + end_date_rev + ".csv?" + params #урл составлен!
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
            result = requests.get(url, headers=headers)
            finamDf=pd.read_csv(io.StringIO(result.text))
            finamDf.drop(['<PER>','<TIME>','<OPEN>','<HIGH>','<LOW>','<LOW>','<VOL>'], axis=1, inplace=True)
            finamDf.rename(columns={"<TICKER>": "SECID", "<DATE>": "TRADEDATE","<CLOSE>": "CLOSE"}, inplace=True)
            finamDf.set_index('TRADEDATE', inplace=True)
            finamDf.index=pd.to_datetime(finamDf.index, format = '%Y%m%d')
            
            if returnClose==False:
                return finamDf
            else:
                return finamDf['CLOSE'].iloc[-1]
        
sampleQuoteObj=FinamQuoteProvider()
print(sampleQuoteObj.getQuotes('SBER',date(2020,10,9),date(2022,10,9),returnClose=True))
sampleFinamDf = sampleQuoteObj.getQuotes('SBER')
print(sampleFinamDf)
        

101.5
           SECID   CLOSE
TRADEDATE               
2021-10-11  SBER  387.60
2021-10-12  SBER  377.79
2021-10-13  SBER  378.80
2021-10-14  SBER  376.09
2021-10-15  SBER  371.82
...          ...     ...
2022-10-03  SBER  114.35
2022-10-04  SBER  110.55
2022-10-05  SBER  108.51
2022-10-06  SBER  108.24
2022-10-07  SBER  101.50

[233 rows x 2 columns]


In [23]:
sampleFinamDf.equals(sampleFinecDf.loc[:,sampleFinecDf.columns!='BOARDID'])

False